In [1]:
!pip install groq
!pip install -qU langchain_community
!pip install pdfplumber

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 3.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 27.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 27.3 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 437.9/437.9 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.2/60.2 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 54.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 77.9 MB/s eta 0:00:00:00:01


In [2]:
import os

os.environ["GROQ_API_KEY"] = "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"
from groq import Groq

from langchain_community.document_loaders import PDFPlumberLoader

In [3]:
import json

In [4]:
def extract_text_pdf(pdf_path):
        loader = PDFPlumberLoader(pdf_path)
        docs = loader.load()
        text = ''
        for doc in docs:
            text += doc.page_content # Append extracted text
        return text 


In [5]:
system_prompt = """You are an AI assistant designed to extract structured resume data.
                   Always respond with a strictly valid JSON object. Use `null` for missing values,
                   ensuring compliance with JSON standards. Do not include explanations,
                   comments, or any additional text outside the JSON structure.
                """



In [6]:
human_prompt = """
             **Task:** Extract key information from the following resume text.

            **Resume Text:**
            {context}

            **Instructions:**
            Please extract the following information and format it in a clear structure:

            1. **Contact Information:**
            - Name:
            - Email:
            - Phone Number:
            - Website/Portfolio/LinkedIn:
            - Github Profile:

            2. **Education:**
            - Institution Name:
            - Degree:
            - Graduation Date:

            3. **Experience:**
            - Job Title:
            - Company Name:
            - Location:
            - Dates of Employment:
            - Description:

            5. **Skills:**
            - Skills:

            **Question:**
            Extract this information as a structured and valid JSON object. Use `null` for missing or unavailable valuesDo not include explanations,
                   comments, or any additional text outside the JSON structure.
        """


In [20]:
pdf_path = "/kaggle/input/reranker-rs/17823436.rank5.pdf"

In [21]:
context = extract_text_pdf(pdf_path)

In [22]:
client = Groq()
completion = client.chat.completions.create(
    model="llama-3.3-70b-versatile",
    messages=[
        {
            "role": "system",
            "content": system_prompt
        },
        {
            "role": "user",
            "content": human_prompt.format(context= context)
        }
    ],
    temperature=1,
    max_completion_tokens=1024,
    top_p=1,
    stream=True,
    stop=None,
)
import json

# Your existing code to get the completion
output = ""
for chunk in completion:
    content = chunk.choices[0].delta.content or ""
    print(content, end="")
    output += content

# Extract the JSON content between ``` markers
json_start = output.find('{')  # Find first curly brace
json_end = output.rfind('}') + 1  # Find last curly brace
json_content = output[json_start:json_end]

# Save the cleaned JSON to file
try:
    json_data = json.loads(json_content)  # Validate it's proper JSON
    with open('resume_data_pdf5.json', 'w') as f:
        json.dump(json_data, f, indent=2)
    print("\nJSON saved successfully to resume_data.json")
except json.JSONDecodeError as e:
    print(f"Error parsing JSON: {e}")
    # Fallback: save the raw output
    with open('raw_output.json', 'w') as f:
        f.write(output)

```json
{
  "contactInformation": {
    "name": "Aaditya Vijay Hirurkar",
    "email": null,
    "phoneNumber": null,
    "websitePortfolioLinkedIn": null,
    "githubProfile": null
  },
  "education": {
    "institutionName": "University of Mumbai",
    "degree": "Bachelor of Engineering, Information Technology",
    "graduationDate": "2008"
  },
  "experience": [
    {
      "jobTitle": "Business Analyst Sr. Technical Business Analyst",
      "companyName": "Company Name",
      "location": null,
      "datesOfEmployment": "Jul 2011 to Dec 2013",
      "description": "Requirement Gathering, Requirement Analysis, Release management, Product management, Product implementation role, BRS analysis, Feasibility analysis, Vender management, Client handling, Product Management"
    },
    {
      "jobTitle": "Software Implementation Engineer",
      "companyName": "Base Information Management Pvt. Ltd.",
      "location": null,
      "datesOfEmployment": "Dec 2008 to Jul 2011",
      "descri